In [5]:
import pytest

# How to write Bioscrape tests

Bioscrape is currently tested with integration tests using [Pytest](https://docs.pytest.org/en/latest/). In general, determining correctness-of-results of complex simulation is somewhere between non-trivial and impossible, and bioscrape tests don't (usually) try to prove correctness. The primary goals of most bioscrape tests are to **catch unintended changes in behavior** and to **ensure that old bugs stay squashed**.

The simplest way to **catch unintended changes in behavior** in a simulation method is to to simply check whether the output of the method is identical to the last time that method was run. Bioscrape has test utilities that make it easy to generate these kinds of tests (as long as the output of your model is some number of numpy arrays). See **How to use bioscrape's test_utils** below. 

# When you should write a test

1. Any time you add a major new feature (e.g., simulator type, reaction mechanism, etc).

2. Any time you catch a bug, *especially* if that bug was caused by a "fix" to the code base that someone might accidentally run into in the future.

# How to write a Pytest test

Pytest is a pip-installable Python testing framework. For detailed documentation on Pytest, see [Pytest's website](https://docs.pytest.org/en/latest/).

In brief, pytest searches bioscrape's directories for any python files starting with `test_` or ending with `_test.py`, searches those files for any functions starting or ending with `test`, and runs those functions. Each such function is considered one test. Tests are expected to contain `assert` statements (ideally only one, but multiple assertions are often appropriate). If those `assert` statements all come back True, the test passes; if any do not, that test fails. 

Here's an example, taken from the Pytest docs. If you want to write a test to **ensure old bugs stay squashed**, you'll probably need to write some of these.

In [1]:
# content of test_sample.py
def inc(x):
    return x + 1


def test_answer():
    assert inc(3) == 5

To run this test, navigate to any folder containing `test_sample.py` and run `pytest`. You should get something like:

```
$ pytest
=========================== test session starts ============================
platform linux -- Python 3.x.y, pytest-5.x.y, py-1.x.y, pluggy-0.x.y
cachedir: $PYTHON_PREFIX/.pytest_cache
rootdir: $REGENDOC_TMPDIR
collected 1 item

test_sample.py F                                                     [100%]

================================= FAILURES =================================
_______________________________ test_answer ________________________________

    def test_answer():
>       assert inc(3) == 5
E       assert 4 == 5
E        +  where 4 = inc(3)

test_sample.py:6: AssertionError
========================= short test summary info ==========================
FAILED test_sample.py::test_answer - assert 4 == 5
============================ 1 failed in 0.12s =============================```

bioscrape's tests are found in the `tests` folder under the main directory. 

Pytest has a number of handy features for conveniently making lots of tests. One such helpful feature you may wish to use is *parameterzation*. This lets you re-use code for multiple tests with the same structure but slightly different inputs -- if you ever find yourself writing a test in which you loop over a bunch of possible inputs, use parameterzation instead. 

DON'T do this:

In [3]:
def test_answer():
    for x in [1,2,4,8,16]:
        assert inc(x) == x+1

Why not? A Pytest test will fail and exit as soon as it hits its first incorrect assertion. If you loop over parameters and test each one in the same test, then the first time it runs into an assertion error, it will quit out and you won't know whether the rest of the tests passed or not. 

Instead do this:

In [6]:
@pytest.mark.parametrize('x', [1,2,4,8,16])
def test_answer(x):
    assert inc(x) == x+1

That `@pytest.mark.parametrize` (note the spelling) will cause Pytest to call `test_answer` five times, as five separate tests, using `x=1` the first time, `x=2` the second time, etc. This way, all five tests will always run, so if something fails, you'll have a good idea of much of your code is broken. You can parametrize multiple inputs this way:

In [8]:
@pytest.mark.parametrize('x,y', [(1,2),(2,3),(4,5),(8,9),(16,17)])
def test_answer(x, y):
    assert add(x, y) == x+1

This will run test_answer once with inputs 1 and 2, again with inputs 2 and 3, etc.

# How to use bioscrape's test_utils

If you want to write a test to **catch unintended changes in behavior**, then you probably want to use one of the tools in bioscrape's `tests/test_utils.py`. These are not installed with bioscrape; they are meant for back-end use only. 

### Checking model outputs

`test_utils`'s most important function is `check_sim_results`. Use this function any time you want to check that a bioscrape model produces consistent output between updates. This ensures that you don't accidentally change core functionality of bioscrape. 

`check_sim_results` expects each test to produce some number of numpy arrays as output. It will check to make sure these numpy arrays are the same as the last time that test was run and the results "frozen". Each test is expected to have a name (a string) and one or more numpy arrays (the outputs of some models, presumably), each keyed by its own (string) name. `check_sim_results` will check for files matching the names of each numpy array in the folder `tests/frozen_sim_results/<test_name>`. If each numpy array matches its stored frozen version, its assertions will succeed and the corresponding test will pass. Otherwise, it will fail. 

If `check_sim_results` *can't* find a file matching any of its numpy arrays, it will **assume the test array is correct** and store that as the new frozen result (after raising a warning to let you know something was funky). 

So, how do you use this to write a new test? 

First, import the contents of `tests/test_utils.py`. Make a new function with `test` in the name, in a file with `test` in its name, somewhere in the `tests` folder. Inside this function: 
1. Set a random seed with test_utils.set_seed(seed). If you don't set a random seed, then any stochastic tests will always fail!
2. Have your function produce simulation output using whatever you bioscrape functionality you want to test. 
3. Still inside your test function, package the numpy arrays you get out as a result in a dictionary `results_dict` of the form `<(str) array name> -> <(np.ndarray)array result>`.
4. Call `test_utils.check_sim_results(test_name, results_dict)`, where `test_name` is a string identifying the test.

Minimal example:

In [1]:
# Contents of bioscrape/tests/not_a_real_test.py
import test_utils
from bioscrape.simulator import py_simulate_model
from bioscrape.types import Model

def trivial_test():
    test_utils.set_seed(54173)
    
    M1 = Model([(['A'], ['B'], 'massaction', {"k":1})])
    AB_results = py_simulate_model(np.linspace(0,100,1000), Model = M1)
    
    M2 = Model([(['C'], ['D'], 'massaction', {"k":10})])
    CD_results = py_simulate_model(np.linspace(0,100,1000), Model = M2)
    
    results_dict = {
        "A_to_B": AB_results,
        "C_to_D": CD_results
    }
    
    test_utils.check_sim_results("my_trivial_test", results_dict)

If you ever want to intentionally change the functionality of a bioscrape component, you'll need to reset any corresponding tests so that they match the new expected behavior. To do this, delete any folders corresponding to tests whose results are affected by the change, then run `pytest` once to populate the few frozen results.

### Checking model SBML I/O

`test_utils` has a convenience function for testing SBML I/O, `check_sbml_IO`, which works much like `check_simulation_results`. `check_sbml_IO` will check that when your model writes an SBML file, it writes exactly the same SBML file as a frozen reference. It will also check that bioscrape can read the frozen SBML file and get back the same model. 

Like with `check_simulation_results`, you'll have to run your test once before actually testing to generate a frozen result, and like `check_simulation_results`, it cannot test correctness of SBML I/O -- only consistency with past versions.

To use `check_sbml_IO`, import `test_utils` and make a new function with `test` in the name, in a file with `test` in its name, somewhere in the `tests` folder. Inside this function: 

1. Set a random seed with test_utils.set_seed(seed). If you don't set a random seed, then any stochastic tests will always fail!
2. Create any number of Models with whatever feature whose I/O you want to test. 
3. Make a dictionary `model_dict` of the form `<(str) model name> -> <Model object>` for each of your models.
4. Call `test_utils.check_sbml_IO(test_name, model_dict)`, where `test_name` is a string identifying the test.

Minimal example:

In [2]:
# Contents of bioscrape/tests/not_a_real_test.py
import test_utils
from bioscrape.types import Model

def trivial_test():
    test_utils.set_seed(54173)
    
    M1 = Model([(['A'], ['B'], 'massaction', {"k":1})])    
    M2 = Model([(['C'], ['D'], 'massaction', {"k":10})])
    
    model_dict = {
        "A_to_B": M1,
        "C_to_D": M2
    }
    
    test_utils.check_sbml_IO("trivial_sbml_test", model_dict)